# Browsing Agent
This notebook will use the internet to download content like pdfs. Then an LLM will decide which files to keep. Next it will create a vector index of the files, for easy RAG.

In [2]:
#imports
%load_ext autoreload
%autoreload 2
import os
from open_agent import OpenAgent
from config import Config
from IPython.display import display
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import requests
from googlesearch import search
import PyPDF2
import faiss
import regex as re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Function to search for PDF URLs using a query
def search_pdf_urls(query, num_results=10):
    pdf_urls = []
    for url in search(query, num_results=num_results):
        # Optionally check if the URL really ends with '.pdf'
        if url.lower().endswith(".pdf"):
            pdf_urls.append(url)
    return pdf_urls

In [4]:
# Function to download a PDF file from a given URL
def download_pdf(url, dest_folder="downloads"):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded: {local_filename}")
        return local_filename
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

In [5]:
# Function to extract metadata (e.g., creation date) from the PDF
def extract_pdf_metadata(pdf_path):
    metadata = {}
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            metadata = reader.metadata
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return metadata

In [6]:
# Function to extract text from the first few pages of the PDF
def extract_pdf_text(pdf_path, max_pages=3):
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            num_pages = len(reader.pages)
            pages_to_read = min(num_pages, max_pages)
            for i in range(pages_to_read):
                page = reader.pages[i]
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text

In [7]:
agent = OpenAgent(api_key=Config.api_key)

In [8]:
def analyze_pdf(pdf_text, context):
    # Analyze the pdf text together with the context.
    # Next we try to see if it is relevant to the context or not
    # Build our XML prompt
    prompt = f"""
        <prompt>
            <objective>Given the following document text as extracted from a PDF, decide if it is relevant to the context or not</objective>
            <instuctions>
                <instruction>
                    Analyze the document text in the section "document_text" and the context in the section "context" and decide if the document is relevant to the context or not.
                </instruction>
                <instruction>
                    It is important that the brand name is the same in the document and the context.
                </instruction>
                <instruction>
                    Answer with "True" if the document is relevant to the context, and "False" if it is not.
                </instruction>
            </instuctions>
            <document_text>
                {pdf_text}
            </document_text>
            <context>
                {context}   
            </context>

        </prompt>"""
    
    # Send the prompt to the OpenAI API
    response = agent.chat(text=prompt)
    # Try to extract the response as True or False
    try:
        output = response.lower()
        if output == "true":
            return True
        elif output == "false":
            return False
    except Exception as e:
        print(f"Error extracting response: {e}")


 

In [9]:
brand = "mölnycke mepilex border flex"
#brand = "mölnycke mepilex border"
#brand = "mölnycke Mepilex Border Heel"
#brand = "Essity Libero Touch"
product_type = "product sheet"


In [10]:
# Construct the search query. The filetype operator helps to target PDFs.
query = f"{brand} {product_type} filetype:pdf"
print("Searching for PDFs...")
pdf_urls = search_pdf_urls(query, num_results=10)
print(f"Found {len(pdf_urls)} PDF URLs.")

relevant_pdfs = []
# Process each found PDF
for url in pdf_urls:
    print(f"\nProcessing URL: {url}")
    file_path = download_pdf(url)
    if not file_path:
        continue
    
   

Searching for PDFs...
Found 8 PDF URLs.

Processing URL: https://www.molnlycke.com/globalassets/mepilex-border-flex-hqim005406.pdf
Downloaded: downloads\mepilex-border-flex-hqim005406.pdf

Processing URL: https://www.molnlycke.ca/contentassets/3a0ad2ec58d848169ae7391bb4370689/mepilexborderflex_productsheet_eng-1.pdf
Downloaded: downloads\mepilexborderflex_productsheet_eng-1.pdf

Processing URL: https://www.onemed.se/-/media/onemed/b2b/ligu/molnlycke/produktblad---mepilex-border-flex.pdf
Downloaded: downloads\produktblad---mepilex-border-flex.pdf

Processing URL: https://static.webareacontrol.com/CommonFile/mepilexborderflex-product-sheet-1647325121893.pdf
Downloaded: downloads\mepilexborderflex-product-sheet-1647325121893.pdf

Processing URL: https://www.onemed.se/-/media/onemed/b2b/ligu/molnlycke/broschyr---mepilex-border-flex.pdf
Downloaded: downloads\broschyr---mepilex-border-flex.pdf

Processing URL: https://briggatemedical.com/wp-content/uploads/wpallimport/files/resource/MepilexB

In [11]:
# Extract the pdf files from the downloads folder
folder = "downloads"
# append the folder to the file path
pdf_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".pdf")]
pdf_files

['downloads\\860717d3e8b7e304716c715e0e22f787.pdf',
 'downloads\\broschyr---mepilex-border-flex.pdf',
 'downloads\\Mepilex-Border-Flex-Family-Product-Sheet.pdf',
 'downloads\\mepilex-border-flex-hqim005406.pdf',
 'downloads\\mepilexborderflex-product-sheet-1647325121893.pdf',
 'downloads\\MepilexBorderFlex.pdf',
 'downloads\\mepilexborderflex_productsheet_eng-1.pdf',
 'downloads\\produktblad---mepilex-border-flex.pdf']

# Filter the PDFs
See if the data is valid for our use case.

In [12]:
company = "Mölnlycke"
#company = "Essity"
context = f"We want to match a product with the brand name: {brand} to the product sheet (pdf). The manufacturer should be {company}. The pdf should be a product sheet."
print(context)
relevant_pdfs = []
for file_path in pdf_files:
    #print(f'Analyzing PDF: {file_path}')
    # Extract metadata such as creation date
    # metadata = extract_pdf_metadata(file_path)
    # creation_date = metadata.get("/CreationDate", "Unknown")
    # print(f"Creation Date (from metadata): {creation_date}")

    # Extract text from the PDF 
    pdf_text = extract_pdf_text(file_path, max_pages=10)

    # See if the PDF is relevant to the context
    is_relevant = analyze_pdf(pdf_text, context)
    print(f"{file_path} Is Relevant: {is_relevant}")
    if is_relevant:
        relevant_pdfs.append(file_path)

We want to match a product with the brand name: mölnycke mepilex border flex to the product sheet (pdf). The manufacturer should be Mölnlycke. The pdf should be a product sheet.
downloads\860717d3e8b7e304716c715e0e22f787.pdf Is Relevant: True
downloads\broschyr---mepilex-border-flex.pdf Is Relevant: True
downloads\Mepilex-Border-Flex-Family-Product-Sheet.pdf Is Relevant: True
downloads\mepilex-border-flex-hqim005406.pdf Is Relevant: True
downloads\mepilexborderflex-product-sheet-1647325121893.pdf Is Relevant: True
downloads\MepilexBorderFlex.pdf Is Relevant: True
downloads\mepilexborderflex_productsheet_eng-1.pdf Is Relevant: True
downloads\produktblad---mepilex-border-flex.pdf Is Relevant: True


# Query the data
Ask questions about the data using the PDFs and an LLM


In [ ]:
# Take the full text of all relevant pdfs and add them in a RAG response
main_query = 'Extract the available sizes from the product sheet, only answer with information that belongs to Mepilex Border. Include the article number for each row. Answer with a table.'
#main_query = 'Extract the available sizes from the product sheet together with their arcticle number. Answer with a table.'
#main_query = f'Summarize the data as a higly informative product description in a few sentenses. Only information relevant to the brand {brand} should be included.'
# Iterate over the relevant PDFs and update the query


files_xml = ""
files_xml = ""
for file_path in relevant_pdfs:
    file_name = os.path.basename(file_path)
    file_content = extract_pdf_text(file_path)
    # Create an XML-like structure for each file.
    files_xml += f"<file>\n  <name>{file_name}</name>\n  <file_content>{file_content}</file_content>\n</file>\n"


query = f"""
    <prompt>
        <objective>Answer the main query by extracting the information from the product sheets. The main quary is in "main_query" and the content is in "content". Each file has its own "file" where "name" is the name of the pdf file, the "file_content" is the actual pdf text.</objective>
        <main_query>
            {main_query}
        </main_query>
        <instruction>
            For each piece of extracted information, provide the source file name.
        </instruction>
        <content>
            {files_xml}
        </content>
    </prompt>
"""

response = agent.chat(text=query)
print(response)

| Article Number | Size (cm) | Size (inch) | Source File Name                                |
|----------------|-----------|--------------|-------------------------------------------------|
| 595211         | 7.5 x 7.5 | 3 x 3        | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 595311         | 10 x 10   | 4 x 4        | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 595011         | 12.5 x 12.5 | 5 x 5      | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 595411         | 15 x 15   | 6 x 6        | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 595611         | 15 x 20   | 6 x 8        | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 583500         | 7.8 x 10  | 3.1 x 4      | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 583300         | 13 x 16   | 5.1 x 6.3    | Mepilex-Border-Flex-Family-Product-Sheet.pdf   |
| 583400         | 15 x 19   | 6 x 7.5      | Mepilex-Border-Flex-Family-Product-Sheet.pdf   | 

This summarizes the available sizes for the Mep

In [15]:
file_content

'•Flex-teknologin gör att förbandet kan sträckas i\nalla riktningar, följer kroppens rörelser och inte\nlossnar i förtid9,10\n•Med Flex-teknologin kan förbandet sitta på\nplats längre tack vare mindre påfrestning på\nkanterna och minskad risk att förbandet\nrullar\xa0sig1,14•Den följsamma sårdynan ger\nökad\xa0patientkomfort1,14\n•Mepilex® Border Flex har bevisad förmåga\natt\xa0hantera mer vätska än något annat\nallt-i-ett skumförband4,5\n•Förbandet är tunt och diskret även när det ärfyllt med sårvätska\n6,10Safetac® mot sårytan\n• Mindre smärta vid förbandsbyte7,11\n• Fastnar inte i såret12,13\n• Sluter kring sårkanterna vilket \nförhindrar maceration7Mepilex® Border Flex\nFlex-teknologi\n• Överlägsen flexibilitet8\nsom gör att förbandet sitter på bättre jämfört med förband utan Flex-teknologi\n9,10Absorberande skumlager\n• Suger omedelbart upp sårvätska och förhindrar läckage, vilket skyddar den omkringliggande huden\n7Baksidesfilm med mycket god andningsbarhet\n• Barriär mot bakter

# Create a vector index of the relevant documents

In [ ]:
text = 'hello world 42'
embedding = agent.get_embedding(text)
print(embedding)

In [ ]:
embedding_dim = len(embedding)
# Create a faiss index
index = faiss.IndexFlatIP(embedding_dim)

In [ ]:
def chunk_text(text, max_length=500):
    """
    Splits text into chunks of up to max_length characters.
    This is a simple splitter that uses sentence boundaries.
    """
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += " " + sentence if current_chunk else sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [ ]:
def embed_text(text, file_name):
    # Split the text into chunks
    chunks = chunk_text(text)
    # Embed each chunk
    embeddings = []
    documents = [] #The documents for each chunk
    for i, chunk in enumerate(chunks):
        if chunk:
            embedding = agent.get_embedding(chunk)
            embeddings.append(embedding)

            documents.append({
                            "file": file_name,
                            "chunk_index": i,
                            "text": chunk
                        })
    return embeddings, documents

In [ ]:
embeddings, documents = embed_text(pdf_text, file_path)

In [ ]:
documents

In [ ]:
# Add the embeddings to the index
embeddings = np.array(embeddings)
index.add(embeddings)

In [ ]:
query = "Available Sizes"
# Embed the query
query_embedding = agent.get_embedding(query)
# Search the index
k = 3
D, I = index.search(np.array([query_embedding]), k)
# Display the search results
for i in range(k):
    print(f"Result {i+1}")
    print(f"Distance: {D[0][i]}")
    print(f"Document: {documents[I[0][i]]['file']}")
    print(f"Chunk Index: {documents[I[0][i]]['chunk_index']}")
    print(f"Text: {documents[I[0][i]]['text']}")
    print()

In [ ]:
relevant_pdfs

In [ ]:
relevant_pdfs = pdf_files